In [ ]:
import os
import datetime
import subprocess
import csv
import smtplib
import zipfile
from email.mime.text import MIMEText
import glob

# ===== CONFIGURATIONS =====
DB_NAME = "facility_db"
DB_USER = "root"
DB_PASS = "your_db_password"

FACILITY_NAME = "facility_A"
BASE_BACKUP_DIR = "/opt/facility_backup/backups/full"
LOG_FILE = "/opt/facility_backup/backup_log.csv"

EMAIL_FROM = "facility.backup.system@gmail.com"
EMAIL_TO = "chipurirowalterc@gmail.com"
EMAIL_PASS = "facility123"

SMTP_SERVER = "smtp.gmail.com"
SMTP_PORT = 587

# ===== FUNCTION: Send Email Alert =====
def send_email_alert(subject, msg):
    message = MIMEText(msg)
    message['Subject'] = subject
    message['From'] = EMAIL_FROM
    message['To'] = EMAIL_TO

    try:
        with smtplib.SMTP(SMTP_SERVER, SMTP_PORT) as server:
            server.starttls()
            server.login(EMAIL_FROM, EMAIL_PASS)
            server.send_message(message)
        print("✅ Email alert sent.")
    except Exception as e:
        print("❌ Failed to send alert email:", e)

# ===== FUNCTION: Perform Backup =====
def perform_backup():
    # Create backup folder
    date_folder = datetime.datetime.now().strftime("%Y-%m-%d")
    backup_dir = os.path.join(BASE_BACKUP_DIR, date_folder)
    os.makedirs(backup_dir, exist_ok=True)

    timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    filename = f"{FACILITY_NAME}_full_{timestamp}.sql"
    filepath = os.path.join(backup_dir, filename)

    # Run mysqldump
    start_time = datetime.datetime.now()
    print(f"🟡 Starting backup: {filepath}")
    with open(filepath, "w") as sql_file:
        result = subprocess.run(
            ["mysqldump", "-u", DB_USER, f"-p{DB_PASS}", DB_NAME],
            stdout=sql_file,
            stderr=subprocess.PIPE
        )
    end_time = datetime.datetime.now()

    # Determine result
    status = "success" if result.returncode == 0 else "failed"
    file_size = os.path.getsize(filepath) / (1024 * 1024) if os.path.exists(filepath) else 0
    duration = (end_time - start_time).seconds
    message = "Backup completed successfully." if status == "success" else result.stderr.decode().strip()

    # Log to CSV
    log_row = [
        timestamp, "full", status, os.path.join(date_folder, filename),
        round(file_size, 2), duration, message
    ]
    file_exists = os.path.exists(LOG_FILE)
    with open(LOG_FILE, "a", newline="") as f:
        writer = csv.writer(f)
        if not file_exists:
            writer.writerow(["timestamp", "type", "status", "filename", "file_size_mb", "duration_sec", "message"])
        writer.writerow(log_row)

    # Handle success/failure
    if status == "failed":
        send_email_alert("🚨 Facility Backup FAILED", f"[{FACILITY_NAME}] Backup failed at {timestamp}.\n\nError:\n{message}")
    else:
        print(f"✅ Backup completed: {filepath} ({round(file_size, 2)} MB in {duration} sec)")

        # ===== ZIP + ENCRYPT the dump =====
        zip_path = filepath + ".zip"
        zip_password = b"YourSecurePassword"  # must be bytes
        with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zf:
            zf.setpassword(zip_password)
            zf.write(filepath, arcname=os.path.basename(filepath))
        print(f"🔐 Encrypted zip created: {zip_path}")

        # Clean up the uncompressed .sql file
        os.remove(filepath)


# Get all zip files in this facility's folder
backup_files = sorted(
    glob.glob(os.path.join(backup_dir, f"{FACILITY_NAME}_full_*.sql.zip")),
    key=os.path.getmtime,
    reverse=True  # newest first
)

# Keep only the 4 newest backups and delete the rest
for old_file in backup_files[4:]:
    os.remove(old_file)
    print(f"Old backup removed: {old_file}")


# ===== MAIN =====
if __name__ == "__main__":
    perform_backup()
